In [49]:
# Dependencies and Setup
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine
import plotly.express as px
from sklearn.cluster import KMeans
import hvplot.pandas


In [50]:
# get password
from config import db_password
from config import user_name
rds_endpoint = 'methanedb.cri33yx04get.us-east-1.rds.amazonaws.com'
port = '5432'
DB_name = 'methaneAnalysis'

url = f'postgresql://{user_name}:{db_password}@{rds_endpoint}:{port}/{DB_name}'

In [51]:
# sql alchemy create_engine postgresql://{user}:{password}@{rds endpoint}:{port}/{DB name}
engine = create_engine(url)

In [52]:
Base = automap_base()
Base.prepare(engine, reflect=True)

In [53]:
# query df = pd.read_sql(query.statement, connection)

# merged table
merged_df = pd.read_sql('SELECT * FROM merged_data', engine)

# sector specific table
sector_total_df = pd.read_sql('SELECT * FROM sector_total', engine)

In [54]:
merged_df.head()

,year,country,sector,emissions,gdp
0,2018,Afghanistan,Agriculture,10.45,18.053229
1,2018,Afghanistan,Energy,67.91,18.053229
2,2018,Afghanistan,Fugitive Emissions,2.22,18.053229
3,2018,Afghanistan,Industrial Processes,0.00,18.053229
4,2018,Afghanistan,Land-Use Change and Forestry,0.00,18.053229


In [55]:
sector_total_df.head()

,year,country,sector,emissions,gdp
0,2018,Afghanistan,Total including LUCF,81.51,18.053229
1,2018,Albania,Total including LUCF,3.16,15.156432
2,2018,Algeria,Total including LUCF,49.55,174.910879
3,2018,Andorra,Total including LUCF,0.05,3.218316
4,2018,Angola,Total including LUCF,39.96,101.353231


In [56]:
# drop world from country column
merged_df = merged_df[merged_df['country'] != 'World']
sector_total_df = sector_total_df[sector_total_df['country'] != 'World']

# ML Model for Sector Dataframe

In [57]:
sector_new = sector_total_df.drop(['country', 'sector'], axis=1)
sector_new.head()

,year,emissions,gdp
0,2018,81.51,18.053229
1,2018,3.16,15.156432
2,2018,49.55,174.910879
3,2018,0.05,3.218316
4,2018,39.96,101.353231


In [58]:
inertia = []
k = list(range(1,11))

# Looking for the best K
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(sector_new)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

In [59]:
# Initializing model with K = 2
model = KMeans(n_clusters=2, random_state=5)
model

# Fitting model
model.fit(sector_new)

KMeans(n_clusters=2, random_state=5)

In [60]:
# Add a new class column to the df
sector_total_df["class"] = model.labels_
sector_total_df.head()

,year,country,sector,emissions,gdp,class
0,2018,Afghanistan,Total including LUCF,81.51,18.053229,0
1,2018,Albania,Total including LUCF,3.16,15.156432,0
2,2018,Algeria,Total including LUCF,49.55,174.910879,0
3,2018,Andorra,Total including LUCF,0.05,3.218316,0
4,2018,Angola,Total including LUCF,39.96,101.353231,0


In [61]:
sector_total_df['scaled'] = (sector_total_df['year'] - 1999) * 1.5
sector_total_df.head()

,year,country,sector,emissions,gdp,class,scaled
0,2018,Afghanistan,Total including LUCF,81.51,18.053229,0,28.5
1,2018,Albania,Total including LUCF,3.16,15.156432,0,28.5
2,2018,Algeria,Total including LUCF,49.55,174.910879,0,28.5
3,2018,Andorra,Total including LUCF,0.05,3.218316,0,28.5
4,2018,Angola,Total including LUCF,39.96,101.353231,0,28.5


In [62]:
# plotting the clusters 
sector_total_df.hvplot.scatter(x='gdp', y='emissions', by='class', s='scaled', hover_cols= ['year', 'country', 'gdp', 'emissions'])

:NdOverlay   [class]
   :Scatter   [gdp]   (emissions,scaled,year,country)

In [63]:
# plot the clusters with 3 features
fig = px.scatter_3d(
    sector_total_df,
    x='year',
    y='gdp',
    z='emissions',
    color='class',
    symbol='class',
    size='emissions',
    width=800
)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [64]:
sector_total_df.corr()

,year,emissions,gdp,class,scaled
year,1.000000,0.016893,0.060430,0.020055,1.000000
emissions,0.016893,1.000000,0.615639,0.566701,0.016893
gdp,0.060430,0.615639,1.000000,0.904005,0.060430
class,0.020055,0.566701,0.904005,1.000000,0.020055
scaled,1.000000,0.016893,0.060430,0.020055,1.000000


In [66]:
plot = sector_total_df.hvplot.scatter(x='gdp', y='emissions', by='class', s='scaled', hover_cols= ['year', 'country', 'gdp', 'emissions'])
hvplot.save(plot, 'resources/plot.html')

fig.write_html("resources/3d.html")